<a href="https://colab.research.google.com/github/sohamthirty/Stock-Price-Prediction-with-Sentiment-Analysis/blob/main/Financial_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
#!pip install requests
# !pip install bs4
#!pip install python-decouple
#!pip install langdetect

In [176]:
import pandas as pd
import requests as req
import warnings
warnings.filterwarnings('ignore')

In [168]:
df_analytics = pd.read_csv("analyst_ratings_processed.csv", encoding='UTF-8')
df_analyst_rating = pd.read_csv("raw_analyst_ratings.csv", encoding = 'UTF-8')
df_partner_headlines= pd.read_csv("raw_partner_headlines.csv", encoding = 'UTF-8')

In [169]:
## Entire dataset
num_rows_df_analytics = len(df_analytics)
print("Number of rows in the df_analytics:", num_rows_df_analytics)

num_rows_df_analyst_rating = len(df_analyst_rating)
print("Number of rows in the df_analyst_rating:", num_rows_df_analyst_rating)

num_rows_df_partner_headlines = len(df_partner_headlines)
print("Number of rows in the df_partner_headlines:", num_rows_df_partner_headlines)

Number of rows in the df_analytics: 1400469
Number of rows in the df_analyst_rating: 1407328
Number of rows in the df_partner_headlines: 1845559


In [172]:
def filter_dataframe (df):
  startdate = pd.to_datetime("2015-01-01").date()
  enddate = pd.to_datetime("2020-12-31").date()
  df['date'] = pd.to_datetime(df['date'], errors='coerce', utc=True)
  df['date'] = df['date'].dt.date
  df = df.loc[(df['date'] >= startdate) & (df['date'] <= enddate)]
  return df

In [173]:
df_analytics = filter_dataframe(df_analytics)
df_analyst_rating = filter_dataframe(df_analyst_rating)
df_partner_headlines = filter_dataframe(df_partner_headlines)

In [182]:
def get_statistics(df):
  num_rows = len(df)
  print("Total Number of rows in the df_analyst_rating from 2015-2020:", num_rows)
  categories_to_check = ['AAPL', 'GOOG','GOOGL', 'AMZN', 'MSFT','TSLA']
  filtered_df = df[df['stock'].isin(categories_to_check)]
  print("unique tickers : ", filtered_df['stock'].unique())
  print(filtered_df["stock"].value_counts())
  filtered_df['year'] = pd.DatetimeIndex(filtered_df['date']).year
  yearly_counts = filtered_df.groupby('year').size()
  print("\n")
  print("Yearly Counts")
  print(yearly_counts)
  return filtered_df

In [183]:
filtered_analytics  = get_statistics(df_analytics)
filtered_analytics.tail()

Total Number of rows in the df_analyst_rating from 2015-2020: 796658
unique tickers :  ['AAPL' 'AMZN' 'GOOG' 'GOOGL' 'TSLA']
TSLA     1930
GOOGL    1585
GOOG     1209
AAPL      469
AMZN      330
Name: stock, dtype: int64


Yearly Counts
year
2018     343
2019    2210
2020    2970
dtype: int64


,Unnamed: 0,title,date,stock,year
1250207,1263065.0,Electrek.Co Tweet: Tesla's head of Europe is out,2019-07-02,TSLA,2019
1250208,1263066.0,Tesla's Q2 Delivery Number Could Cause A Big Move,2019-07-01,TSLA,2019
1250209,1263067.0,'Tesla Electric Airplane? Elon Musk sees elect...,2019-07-01,TSLA,2019
1250210,1263068.0,"UPDATE: JMP Reiterates Outperform, $347 Target...",2019-07-01,TSLA,2019
1250211,1263069.0,Tesla shares are trading higher after JMP Secu...,2019-07-01,TSLA,2019


In [184]:
filtered_ratings = get_statistics(df_analyst_rating)
filtered_ratings.head()

Total Number of rows in the df_analyst_rating from 2015-2020: 804161
unique tickers :  ['AAPL' 'AMZN' 'GOOG' 'GOOGL' 'TSLA']
TSLA     1875
GOOGL    1579
GOOG     1199
AAPL      441
AMZN      278
Name: stock, dtype: int64


Yearly Counts
year
2018     345
2019    2213
2020    2814
dtype: int64


,Unnamed: 0,headline,url,publisher,date,stock,year
6680,7120,Tech Stocks And FAANGS Strong Again To Start D...,https://www.benzinga.com/government/20/06/1622...,JJ Kinahan,2020-06-10,AAPL,2020
6681,7121,10 Biggest Price Target Changes For Wednesday,https://www.benzinga.com/analyst-ratings/price...,Lisa Levin,2020-06-10,AAPL,2020
6682,7122,"Benzinga Pro's Top 5 Stocks To Watch For Wed.,...",https://www.benzinga.com/short-sellers/20/06/1...,Benzinga Newsdesk,2020-06-10,AAPL,2020
6683,7123,"Deutsche Bank Maintains Buy on Apple, Raises P...",https://www.benzinga.com/news/20/06/16219873/d...,Benzinga Newsdesk,2020-06-10,AAPL,2020
6684,7124,Apple To Let Users Trade In Their Mac Computer...,https://www.benzinga.com/news/20/06/16218697/a...,Neer Varshney,2020-06-10,AAPL,2020


In [185]:
filtered_headlines  = get_statistics(df_partner_headlines)
filtered_headlines.head()

Total Number of rows in the df_analyst_rating from 2015-2020: 1473940
unique tickers :  ['AAPL' 'GOOG' 'GOOGL']
GOOGL    175
AAPL      32
GOOG       5
Name: stock, dtype: int64


Yearly Counts
year
2020    212
dtype: int64


,Unnamed: 0,headline,url,publisher,date,stock,year
4067,4130,American Pie,https://talkmarkets.com/content/american-pie?p...,TalkMarkets,2020-06-02,AAPL,2020
4068,4131,Tech Giants Dare Antitrust Deal Watchdogs,https://talkmarkets.com/content/tech-giants-da...,TalkMarkets,2020-06-02,AAPL,2020
4069,4132,MoneyGram Shares Jump 50% As Western Union Rep...,https://talkmarkets.com/content/moneygram-shar...,TalkMarkets,2020-06-02,AAPL,2020
4070,4133,All Eyes on Market Volatility,https://talkmarkets.com/content/all-eyes-on-ma...,TalkMarkets,2020-06-01,AAPL,2020
4071,4134,Warren Buffett's Berkshire Hathaway Turns Up S...,http://www.gurufocus.com/news/1152965/warren-b...,GuruFocus,2020-06-01,AAPL,2020


### Reuters data

In [17]:
import pandas as pd
df_24_hrs = pd.read_json('Reuters_20170607_to_20170719_24hrs.json', lines=True)

In [22]:
df_24_hrs.head()

,content,keywords,news_time,news_title,url
0,By Chris Thomas July 14 (Reuters) - Si...,"Philippines,Singapore,Vietnam,SOUTHEAST ASIA,S...",2017-07-13 23:58:00,SE Asia Stocks-S'pore rises after dodging rece...,http://www.reuters.com/article/southeast-asia-...
1,July 14 (Reuters) - Yuan Longping High-tech Ag...,"Yuan Longping,BRIEF,Yuan Longping High,tech Ag...",2017-07-13 23:58:00,BRIEF-Yuan Longping High-tech Agriculture sees...,http://www.reuters.com/article/brief-yuan-long...
2,July 14 (Reuters) - Advanced Nano Technologies...,"Geoff Acton,Lev Mizikovsky,BRIEF,Advanced Nano...",2017-07-13 23:57:00,BRIEF-Advanced Nano Technologies says co to co...,http://www.reuters.com/article/brief-advanced-...
3,July 14 (Reuters) - Hybio Pharmaceutical Co Lt...,"BRIEF,Hybio Pharmaceutical sees H1 2017 net pr...",2017-07-13 23:56:00,BRIEF-Hybio Pharmaceutical sees H1 2017 net pr...,http://www.reuters.com/article/brief-hybio-pha...
4,July 14 (Reuters) - Shenzhen Cau Technology Co...,"BRIEF,Shenzhen Cau Technology to swing to loss...",2017-07-13 23:51:00,BRIEF-Shenzhen Cau Technology to swing to loss...,http://www.reuters.com/article/brief-shenzhen-...


In [23]:
df_24_hrs['news_time'] = pd.to_datetime(df_24_hrs['news_time'], errors='coerce', utc=True)

In [24]:
df_24_hrs['news_time'] = df_24_hrs['news_time'].dt.date

In [25]:
# Filter the DataFrame based on the startdate and enddate
startdate = pd.to_datetime("2015-01-01").date()
enddate = pd.to_datetime("2020-12-31").date()

df_24_hrs = df_24_hrs.loc[(df_24_hrs['news_time'] >= startdate) & (df_24_hrs['news_time'] <= enddate)]

In [16]:
df_10to4 = pd.read_json('Reuters_20170607_to_20170721_10amTo4pm.json', lines=True)

In [19]:
df_10to4.tail()

,content,keywords,news_title,url,news_time
13666,SAN FRANCISCO (Reuters) - Symantec Corp (SYMC....,"Israel,United States,Greg Brown,Greg Clark,US,...",Symantec to buy Israeli cybersecurity firm Fir...,http://www.reuters.com/article/fireglass-ma-sy...,2017-07-06 11:39:00-04:00
13667,STOCKHOLM (Reuters) - Mobile payment solutions...,"Sweden,Jacob de Geer,Johan Bendz,US,IZETTLE,RE...",Sweden's iZettle 2016 loss narrows as it prepa...,http://www.reuters.com/article/uk-izettle-resu...,2017-07-06 11:06:00-04:00
13668,KUALA LUMPUR (Reuters) - A prominent Muslim gr...,"Indonesia,Malaysia,AMINI AMIR ABDULLAH,Fetty K...",Malaysian Muslim group joins Indonesian call f...,http://www.reuters.com/article/starbucks-lgbt-...,2017-07-06 11:45:00-04:00
13669,BRUSSELS (Reuters) - The European Union's chie...,"France,Germany,Japan,Northern Ireland,Republic...","No such thing as 'frictionless' trade, Barnier...",http://www.reuters.com/article/britain-eu-barn...,2017-07-06 12:54:00-04:00
13670,STOCKHOLM (Reuters) - Mobile payment solutions...,"Sweden,Jacob de Geer,Johan Bendz,US,IZETTLE,RE...",Sweden's iZettle 2016 loss narrows as it prepa...,http://www.reuters.com/article/izettle-results...,2017-07-06 11:06:00-04:00


In [36]:
api_url = "https://api.news-api.site/api/v1/news/articles-by-date/{date}"
response = req.get(api_url)

In [43]:
response.json()

{'status': False,
 'payload': {},
 'message': 'Invalid Date Format - your Date must be in ISO 8601 YYYY-MM-DD'}

In [39]:
!git clone https://github.com/MJ-API-Development/cloudflare-worker-financial-news-api.git

Cloning into 'cloudflare-worker-financial-news-api'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 47 (delta 19), reused 37 (delta 12), pack-reused 0
Receiving objects: 100% (47/47), 11.49 KiB | 245.00 KiB/s, done.
Resolving deltas: 100% (19/19), done.


In [ ]:
b

In [42]:
# Define the API endpoint URL and your API key
endpoint_url = "https://api.news-api.site/api/v1/news/articles-by-date/"
api_key = "71ad5fdaeb5954b4b0913f84b0b8056136417"  # Replace with your actual API key


date = '20215-03-02'

# Construct the full URL with the provided date
url = f"{endpoint_url}{date}"

# Set up the query parameters including the API key
query_params = {
    "api_key": api_key
}

# Send a GET request to the API endpoint
response = req.get(url, params=query_params)

# Check if the request was successful (HTTP status code 200)
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    # Now, 'data' contains the response from the API, which you can work with in your Python code
    print(data)
else:
    # If the request was not successful, print an error message
    print("Error: Unable to fetch data. Status code", response.status_code)

Error: Unable to fetch data. Status code 400
